In [5]:
import pickle
import pandas as pd
from datetime import datetime, date, time, timedelta
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textrank
import os
import sys
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict
from math import log, floor
import numpy as np
import textrank
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, recall_score, precision_score, precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
import pickle
import os
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
stop_words = stopwords.words("english")
stop_words.extend(["chris", "satterfield"])

In [6]:
total_counter = None

def tfidf_cosine_similarity(task, description):
    global total_counter
    c = Counter(description)
    for key in c:
        if key in total_counter:
            c[key] = c[key]/total_counter[key]
            
    v1 = []
    v2 = []
    for key in total_counter:
        if key in task:
            v1.append(task[key])
        else:
            v1.append(0)
            
        if key in c:
            v2.append(c[key])
        else:
            v2.append(0)
            
            
    return sim(np.array(v1), np.array(v2))
    
    

def rake(snapshot):
    r = Rake()
    r.extract_keywords_from_text(snapshot)
    keyphrases = r.get_ranked_phrases()
    tokens = []
    [tokens.extend(x.split(" ")) for x in keyphrases]
    return tokens

def valid(dateobj, start, end):
    return dateobj >= start and dateobj < end        

def tfidf(task_counters, weighted_df):
    global total_counter
    idf_counters = []
    for counter in task_counters:
        n = Counter()
        for key in counter:
            n[key] = 1
        idf_counters.append(n)
    total_counter = sum(idf_counters, Counter())
    
    weighted_counters = []
    for counter in task_counters:
        occurances = []
        for key in counter.keys():
            occurances.append(total_counter[key])
        
        mean = np.mean(occurances)
        
        weighted = {}
        for key in counter.keys():
            if(weighted_df):
                weighted[key] = counter[key]/((total_counter[key])  * (1 + abs(mean - total_counter[key])))
            else:
                weighted[key] = counter[key] /(total_counter[key])
        weighted_counters.append(Counter(weighted))
    return weighted_counters


In [7]:
cache = {}

In [25]:
def shuffleDict(dictionary):
    keys = list(dictionary.keys())
    random.shuffle(keys)
    shuffled = {}
    for key in keys:
        shuffled[key] = dictionary[key]
    
    return shuffled

def equals(prediction, expected):
    if(prediction == expected):
        return 1
    else:
        return 0

def get_prediction(scores):
    scores = shuffleDict(scores)
    pred = max(scores, key=scores.get)
    #choices = list(range(1,len(scores) + 1))
    #choices.append(-1)
    #return np.random.choice(choices)
    #if(scores[pred] < 0.05):
    #    return -1
    #else:
    return pred

def normalize_score(scores):
    norm = np.linalg.norm(list(scores.values()), ord=1)
    for score in scores:
        if(norm != 0):
            scores[score] = scores[score] / norm
    return scores

class WindowTitleTaskExtractor:

    def get_tasks_for_participant(self, path_to_data, participant, tokenizer, weighted=False, without_emails=True, using_tfidf=False, ungrouped=False, widf=False):
        
        appdata = pd.read_csv(f"{path_to_data}/{participant}/appdata_fixed.csv")
        df = pd.read_excel(f"{path_to_data}/{participant}/taskswitches_annotated.xlsx")
        offset = df[df["task"] == "offset"]["end"].iloc[0]
        startDelta = datetime.combine(date.min, offset) - datetime.min 
        df = df[df["task"] != "offset"]
        task_words_ungrouped = []
        task_order = []
        

        studyStartTime = appdata["StartTime"][0] - (datetime.combine(date.min, offset) - datetime.min).total_seconds()
        start = studyStartTime

        for _,row in df.iterrows():
            startDelta = datetime.combine(date.min, row["start"]) - datetime.min 
            endDelta = datetime.combine(date.min, row["end"]) - datetime.min
            start = studyStartTime + startDelta.total_seconds()
            taskEnd = studyStartTime + endDelta.total_seconds()
            while(start < taskEnd):
                end = start + 60
                titles = appdata[((appdata["StartTime"] >= start) & (appdata["StartTime"] < end)) | ((appdata["EndTime"] >= start) & (appdata["EndTime"] < end))]
                titles = list(titles["WindowTitle"])
                titles = [x.lower() for x in titles]
                start = start + 60


                if(without_emails):
                    titles = [x for x in titles if not "gmail" in x]

                words = []
                for title in titles:
                    tokens = word_tokenize(title)
                    snapshot_words = [x for x in tokens if len(x) > 2]
                    snapshot_words = [stemmer.stem(x) for x in snapshot_words if not x in stop_words]
                    words.extend(snapshot_words)

                if(len(words) > 0):
                    task_words_ungrouped.append(words)
                    task_order.append(row["task"])
                print(words)
        
        task_counters = [Counter(x) for x in task_words_ungrouped]
        if(not ungrouped):
            counters = defaultdict(Counter)
            for counter, task in zip(task_counters, task_order):
                counters[task] += counter
            task_order, task_counters = zip(*counters.items())
            
        if(using_tfidf):
            task_counters = tfidf(task_counters, weighted)

        for task in task_counters:
            norm = np.linalg.norm(list(task.values()))
            for word in task:
                task[word] = task[word]/norm

        return task_counters, task_order

            
            
class ScreenshotTaskExtractor(object):

    def isProbablyEmail(self, task):
        excludedWords = ['compose', 'gmail', 'inbox', 'google', 'starred', 'sent','mail','drafts','more','terms','privacy','program','policies']
        i = 0
        for word in excludedWords:
            if word in task:
                i += 1
        return i > 4

    def get_tasks_for_participant(self, path_to_data, participant, tokenizer, weighted=False, without_emails=True, ungrouped=False, using_tfidf=False):
        with open(f"{path_to_data}/{participant}/fulltext.pkl", "rb") as f:
            snapshotsWithDates = pickle.load(f)
        
        df = pd.read_excel(f"{path_to_data}/{participant}/taskswitches_annotated.xlsx")
        offset = df[df["task"] == "offset"]["end"].iloc[0]
        startDelta = datetime.combine(date.min, offset) - datetime.min 
        df = df[df["task"] != "offset"]
        task_words_ungrouped = []
        task_order = []
        

        studyStartTime = snapshotsWithDates[0][0] - (datetime.combine(date.min, offset) - datetime.min)
        start = studyStartTime

        for _,row in df.iterrows():
            startDelta = datetime.combine(date.min, row["start"]) - datetime.min 
            endDelta = datetime.combine(date.min, row["end"]) - datetime.min
            start = studyStartTime + startDelta
            taskEnd = studyStartTime + endDelta
            while(start < taskEnd):
                end = start + timedelta(seconds=60)
                snapshotsInTask = [x[1].lower() for x in snapshotsWithDates if valid(x[0], start, end)]
                start = start + timedelta(seconds=60)

                if(without_emails):
                    snapshotsInTask = [x for x in snapshotsInTask if not self.isProbablyEmail(x)]

                words = []
                for snapshot in snapshotsInTask:
                    tokens = tokenizer(snapshot)
                    snapshot_words = [x for x in tokens if len(x) > 2]
                    snapshot_words = [stemmer.stem(x) for x in snapshot_words if not x in stop_words]
                    words.extend(snapshot_words)

                if(len(words) > 0):
                    task_words_ungrouped.append(words)
                    task_order.append(row["task"])
        
        task_counters = [Counter(x) for x in task_words_ungrouped]
        if(not ungrouped):
            counters = defaultdict(Counter)
            for counter, task in zip(task_counters, task_order):
                counters[task] += counter
            task_order, task_counters = zip(*counters.items())
            
        if(using_tfidf):
            task_counters = tfidf(task_counters, weighted)

        for task in task_counters:
            norm = np.linalg.norm(list(task.values()))
            for word in task:
                task[word] = task[word]/norm

        return task_counters, task_order
    
    
def sim(v1, v2):
    return cosine_similarity(v1.reshape(1,-1), v2.reshape(1,-1))[0][0]

def create_results(method, predicted, expected, author, participant):
    return [{"method": method, "predicted": x, "expected": y, "author": author, "participant": participant, "correct": equals(x, y)} for x,y in zip(predicted, expected)]

prob = []

def predict_simple(tasks, order, phrases):
    predictions = []
    expected = []
    durations = []
    sim_vocab = []

    for task, actual in zip(tasks, order):
        expected.append(actual)

        scores = dict()
        cover_scores = dict()
        words = []
        cover = {}
        
        for _, row in phrases.iterrows():
            search_terms = word_tokenize(row["phrase"])
            search_terms = [x.lower() for x in search_terms if not x.lower() in stop_words]
            search_terms = [stemmer.stem(x) for x in search_terms]
            
            
            scores[row["expected"]] = tfidf_cosine_similarity(task, search_terms)

        pred = get_prediction(scores)
        predictions.append(pred)
    return predictions, expected



In [26]:
df = pd.read_excel("../phrases_medium_and_high_v2.xlsx")    
results = []
#participants = ["P01", "P02", "P03", "P04", "P05", "P06", "P07", "P08", "P11", "P12", "P13", "P14", "P15", "P16", "P17", "P18", "P19"]
participants = ["P01", "P02", "P03", "P04", "P05", "P06", "P14", "P15", "P16", "P17", "P18", "P19"]

results_tfidf = []
results_tf = []
results_rake = []
results_rake_tfidf = []

path_to_data = "../../archives/"
task_extractor = WindowTitleTaskExtractor()


for participant in participants:
    print("START ----- ")
    print(participant)
    
    
    #if(participant in cache):
    #    tasks,order = cache[participant]
    #else:
    #    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, word_tokenize, using_tfidf=True, ungrouped=True)
    #    cache[participant] = (tasks, order)
    
    
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, word_tokenize, using_tfidf=True, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("tfidf", predicted, expected, author, participant)
        results.extend(r)
    
    '''
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, word_tokenize, using_tfidf=False, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("tf", predicted, expected, author, participant)
        results.extend(r)
        
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, rake, using_tfidf=False, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("rake", predicted, expected, author, participant)
        results.extend(r)
        
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, rake, using_tfidf=True, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("rake-tfidf", predicted, expected, author, participant)
        results.extend(r)
    '''
    print("END -----")
    
y_true = [x["expected"] for x in results]
y_pred = [x["predicted"] for x in results]

print(accuracy_score(y_true, y_pred))
print(precision_recall_fscore_support(y_true, y_pred, labels=[1,2,3,4,5,6]))
print(matthews_corrcoef(y_true, y_pred))

    
    
    

START ----- 
P01
['bugzilla', 'main', 'page']
['2277', 'full', 'text', 'search', 'gloda', 'find', 'mail', '2271', 'messag', 'filter', 'fail', 'run', 'messag', 'move', 'folder', '2268', 'column', 'set', 'open', 'messag', 'convers', 'persist', '2277', 'full', 'text', 'search', 'gloda', 'find', 'mail', '2271', 'messag', 'filter', 'fail', 'run', 'messag', 'move', 'folder', '2268', 'column', 'set', 'open', 'messag', 'convers', 'persist']
['2268', 'column', 'set', 'open', 'messag', 'convers', 'persist', '2277', 'full', 'text', 'search', 'gloda', 'find', 'mail', '2271', 'messag', 'filter', 'fail', 'run', 'messag', 'move', 'folder', '2268', 'column', 'set', 'open', 'messag', 'convers', 'persist']
['2268', 'column', 'set', 'open', 'messag', 'convers', 'persist', 'bug', 'list', '2268', 'column', 'set', 'open', 'messag', 'convers', 'persist', 'bug', 'list', '2268', 'column', 'set', 'open', 'messag', 'convers', 'persist']
['2268', 'column', 'set', 'open', 'messag', 'convers', 'persist', 'bug', 'li

['microsoft', 'to-do', 'app', 'store', 'microsoft', 'to-do', 'app', 'store']
[]
[]
[]
[]
['microsoft', 'to-do', 'app', 'store', 'https', '//www.google.ca/search', 'q=deep+learning+popular', 'oq=deep+learning+popular', 'aqs=chrome..69i57j0l3.5253j0j4', 'sourceid=chrom', 'ie=utf-8', 'deep', 'learn', 'popular', 'googl', 'search', 'tell', 'deep', 'learn', 'popular', 'demand']
[]
['tell', 'deep', 'learn', 'popular', 'demand', 'tell', 'deep', 'learn', 'popular', 'demand', 'tell', 'deep', 'learn', 'popular', 'demand']
['tell', 'deep', 'learn', 'popular', 'demand', 'tell', 'deep', 'learn', 'popular', 'demand']
['tell', 'deep', 'learn', 'popular', 'demand', 'tell', 'deep', 'learn', 'popular', 'demand']
['tell', 'deep', 'learn', 'popular', 'demand', 'new', 'tab', 'https', '//www.google.ca/search', 'q=deep+learning+gpu+vs+cpu', 'oq=deep+learning+gpu+vs+cpu', 'aqs=chrome.0.0l4.8263j0j7', 'sourceid=chrom', 'ie=utf-8', 'deep', 'learn', 'gpu', 'cpu', 'googl', 'search', 'tensorflow', 'perform', 'test'

[]
['differ', 'blockchain', 'distribut', 'ledger', 'technolog', 'blockchain', 'advantag', 'older', 'ledger', 'system', 'googl', 'search', 'blockchain', 'advantag', 'older', 'ledger', 'system', 'googl', 'search', 'blockchain', 'advantag', 'disadvantag', 'nudj', 'medium']
['blockchain', 'advantag', 'disadvantag', 'nudj', 'medium', 'blockchain', 'advantag', 'disadvantag', 'nudj', 'medium', 'blockchain', 'advantag', 'disadvantag', 'nudj', 'medium', 'blockchain', 'advantag', 'disadvantag', 'nudj', 'medium']
['blockchain', 'advantag', 'disadvantag', 'nudj', 'medium', 'https', '//www.google.ca/search', 'q=proof-of-work+blockchain', 'oq=proof-of-work+blockchain+', 'aqs=chrome..69i57j0l5.5616j1j4', 'sourceid=chrom', 'ie=utf-8', 'proof-of-work', 'blockchain', 'googl', 'search', 'proof-of-work', 'blockchain', 'googl', 'search', 'https', '//blockgeeks.com/guides/proof-of-work-vs-proof-of-stak', 'proof', 'work', 'proof', 'stake', 'basic', 'mine', 'guid', 'blockgeek', 'proof', 'work', 'proof', 'stak

END -----
START ----- 
P04
['new', 'tab', 'bugzilla', 'main', 'page', 'new', 'tab', 'bugzilla', 'main', 'page', '2278', 'simpl', 'mapi', 'win7', '32b', "n't", 'work', '47.0b2']
['2278', 'simpl', 'mapi', 'win7', '32b', "n't", 'work', '47.0b2', '2274', 'show', 'status', 'send', 'mail', 'progress', 'status', 'bar', '2278', 'simpl', 'mapi', 'win7', '32b', "n't", 'work', '47.0b2', '2274', 'show', 'status', 'send', 'mail', 'progress', 'status', 'bar', 'simpl', 'search', 'bug', 'list', 'bugzilla', 'main', 'page', 'new', 'tab', 'bugzilla', 'main', 'page']
['bugzilla', 'main', 'page', '2278', 'simpl', 'mapi', 'win7', '32b', "n't", 'work', '47.0b2', '2274', 'show', 'status', 'send', 'mail', 'progress', 'status', 'bar', '2274', 'show', 'status', 'send', 'mail', 'progress', 'status', 'bar', '2278', 'simpl', 'mapi', 'win7', '32b', "n't", 'work', '47.0b2', 'bugzilla', 'main', 'page']
['bugzilla', 'main', 'page', '2278', 'simpl', 'mapi', 'win7', '32b', "n't", 'work', '47.0b2', 'bugzilla', 'main', 'pa

['wunderlist', 'to-do', 'list', 'remind', 'errand', 'app', 'year', 'deep', 'learn', 'wikipedia', 'deep', 'learn', 'histori', 'googl', 'search']
['deep', 'learn', 'histori', 'googl', 'search', 'histori', 'potenti', 'deep', 'learn', 'answer']
['histori', 'potenti', 'deep', 'learn', 'answer']
['histori', 'potenti', 'deep', 'learn', 'answer']
['histori', 'potenti', 'deep', 'learn', 'answer']
['welcom', 'microsoft', 'to-do', 'timelin', 'visual', 'tool', 'research', 'guid', 'harvard', 'librari', 'histori', 'potenti', 'deep', 'learn', 'answer']
['welcom', 'microsoft', 'to-do', 'timelin', 'visual', 'tool', 'research', 'guid', 'harvard', 'librari', 'histori', 'potenti', 'deep', 'learn', 'answer', 'https', '//www.google.ca/search', 'q=day+chart+visualiy', 'oq=day+chart+visualiy', 'aqs=chrome..69i57.18902j0j4', 'sourceid=chrom', 'ie=utf-8', 'day', 'chart', 'visualiy', 'googl', 'search', 'day', 'chart', 'visual', 'googl', 'search']
['day', 'chart', 'visual', 'googl', 'search', 'tableau', '201', 'm

...

['deep', 'learn', 'gpu', 'cpu', 'googl', 'search', 'gpu', 'alway', 'superior', 'respect', 'deep', 'learn', 'applic', 'googl', 'search', 'gpu', 'cpu', 'futur', 'think-tank', 'superior', 'perform', 'commit', 'kyoto', 'univers', 'cpus', 'gpus', 'intel®', 'softwar']
['superior', 'perform', 'commit', 'kyoto', 'univers', 'cpus', 'gpus', 'intel®', 'softwar', 'battl', 'product', 'app', 'todoist', 'vs.', 'wunderlist', 'vs.', 'any.do']
['battl', 'product', 'app', 'todoist', 'vs.', 'wunderlist', 'vs.', 'any.do']
['battl', 'product', 'app', 'todoist', 'vs.', 'wunderlist', 'vs.', 'any.do', 'battl', 'product', 'app', 'todoist', 'vs.', 'wunderlist', 'vs.', 'any.do', 'microsoft', 'to-do', 'interfac', 'googl', 'search', 'battl', 'product', 'app', 'todoist', 'vs.', 'wunderlist', 'vs.', 'any.do', 'microsoft', 'to-do', 'interfac', 'googl', 'search', 'microsoft', 'to-do', 'share', 'googl', 'search', 'introduc', 'list', 'share', 'step', 'microsoft', 'to-do', 'microsoft', 'tech', 'communiti', '200109']
['int

['introduc', 'microsoft', 'to-do—now', 'avail', 'preview', 'microsoft', '365', 'blog', 'introduc', 'microsoft', 'to-do—now', 'avail', 'preview', 'microsoft', '365', 'blog', 'microsoft', 'wunderlist', 'acquisit', 'get', 'complic', 'verg', 'todoist', 'wunderlist', 'microsoft', 'to-do', 'task', 'tool', 'head-to-head', 'todoist', 'wunderlist', 'microsoft', 'to-do', 'task', 'tool', 'head-to-head', 'new', 'tab', '792+944', 'googl', 'search']
['new', 'tab', 'neural', 'network', 'train', 'weight', 'googl', 'search', 'neural', 'network', 'train', 'weight', 'googl', 'search']
['neural', 'network', 'train', 'weight', 'googl', 'search', 'neural', 'network', 'train', 'weight', 'googl', 'search', 'train', 'artifici', 'neural', 'network', 'intro', 'solver']
['train', 'artifici', 'neural', 'network', 'intro', 'solver']
['train', 'artifici', 'neural', 'network', 'intro', 'solver']
['train', 'artifici', 'neural', 'network', 'intro', 'solver', 'deep', 'learn', 'histori', 'googl', 'search', 'forb', 'welco

['gpu', 'cpu', 'deep', 'learn', 'benchmark', 'googl', 'search', 'dee', 'learn', 'netwrok', 'decid', 'weight', 'choos', 'googl', 'search', 'deep', 'learn', 'network', 'train', 'work', 'googl', 'search', 'deep', 'learn', 'work', 'cosmos']
['deep', 'learn', 'work', 'cosmos', 'deep', 'learn', 'network', 'train', 'work', 'googl', 'search', 'train', 'neural', 'network', 'toward', 'data', 'scienc']
[]
['train', 'neural', 'network', 'toward', 'data', 'scienc', 'understand', 'blockchain', 'fundament', 'part', 'proof', 'work', 'proof', 'stake']
['understand', 'blockchain', 'fundament', 'part', 'proof', 'work', 'proof', 'stake', 'train', 'neural', 'network', 'toward', 'data', 'scienc']
['new', 'tab', 'bugzilla', 'main', 'page', '2256', 'request', 'warn', 'user', 'click', 'potenti', 'danger', 'link', 'email', 'new', 'tab']
['bugzilla', 'main', 'page', '2256', 'request', 'warn', 'user', 'click', 'potenti', 'danger', 'link', 'email', 'new', 'tab', 'bugzilla', 'main', 'page', 'bug', 'list']
['2256', 

['python', 'plot', 'static', 'data', 'export', 'googl', 'search', 'new', 'tab', 'python', 'plot', 'static', 'data', 'export', 'googl', 'search', 'new', 'tab', 'distribut', 'ledger', 'tradit', 'client-serv', 'databas', 'ledger', 'system', 'googl', 'search']
['distribut', 'ledger', 'tradit', 'client-serv', 'databas', 'ledger', 'system', 'googl', 'search', 'blockchain', 'versus', 'tradit', 'databas', 'hacker', 'noon']
[]
[]
[]
['differ', 'blockchain', 'databas', 'coindesk', 'central', 'ledger', 'distribut', 'ledger', 'layman', 'understand', 'proof', 'work', 'blockchain', 'googl', 'search', 'explain', 'blockchain', 'proof', 'work', 'enabl', 'trustless', 'consensus']
['explain', 'blockchain', 'proof', 'work', 'enabl', 'trustless', 'consensus', 'explain', 'blockchain', 'proof', 'work', 'enabl', 'trustless', 'consensus']
['explain', 'blockchain', 'proof', 'work', 'enabl', 'trustless', 'consensus', 'app', 'store', 'googl', 'search', 'app', 'store', 'appl', 'wunderlist', 'appl', 'wunderlist', '

END -----
0.5400332225913621
(array([0.68221203, 0.45911047, 0.45802161, 0.33482476, 0.64943457,
       0.63425029]), array([0.66925   , 0.54700855, 0.45916667, 0.37731092, 0.56619718,
       0.56448276]), array([0.67566885, 0.49921997, 0.45859342, 0.35480047, 0.60496614,
       0.59733625]), array([4000, 2340, 3600, 2380, 2840, 2900]))
0.44489807792207015


In [9]:
df = pd.DataFrame(results)

In [245]:
df.to_excel("matching_results.xlsx")